# ScoreHub
In this notebook we are...

In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests

In [3]:
!pip install bs4

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable


In [10]:
from bs4 import BeautifulSoup

## Webscrapping
On the site we see that there is info on 6 European leagues:
* La Liga
* EPL
* BundesLiga
* Serie A
* Ligue 1
* RFPL

We are only interested on gathering data for the EPL league. 

In [11]:
# create urls for all seasons of all leagues
base_url = 'https://understat.com/league'
leagues = ['La_liga', 'EPL', 'Bundesliga', 'Serie_A', 'Ligue_1', 'RFPL']
#seasons = ['2014', '2015', '2016', '2017', '2018']
seasons = ['2018','2019','2020', '2021', '2022']

In [20]:
import lxml 
# Starting with latest data for Spanish league, because I'm a Barcelona fan
url = base_url+'/'+leagues[1]+'/'+seasons[4]
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser') # changed this from 'lxml' to 'html.parser' 

# Based on the structure of the webpage, I found that data is in the JSON variable, under <script> tags
scripts = soup.find_all('script')

# Check our <script> tags
# for el in scripts:
#   print('*'*50)
#   print(el.text)

In [21]:
# Extract the json
import json

string_with_json_obj = ''

# Find data for teams
for el in scripts:
    if 'teamsData' in str(el):
      string_with_json_obj = str(el).strip()
      
# print(string_with_json_obj)

# strip unnecessary symbols and get only JSON data
ind_start = string_with_json_obj.index("('")+2
ind_end = string_with_json_obj.index("')")
json_data = string_with_json_obj[ind_start:ind_end]

json_data = json_data.encode('utf8').decode('unicode_escape')

<script>
	var teamsData = JSON.parse('\x7B\x2271\x22\x3A\x7B\x22id\x22\x3A\x2271\x22,\x22title\x22\x3A\x22Aston\x20Villa\x22,\x22history\x22\x3A\x5B\x7B\x22h_a\x22\x3A\x22a\x22,\x22xG\x22\x3A0.488895000000000023998580900297383777797222137451171875,\x22xGA\x22\x3A0.5883410000000000028563817977556027472019195556640625,\x22npxG\x22\x3A0.488895000000000023998580900297383777797222137451171875,\x22npxGA\x22\x3A0.5883410000000000028563817977556027472019195556640625,\x22ppda\x22\x3A\x7B\x22att\x22\x3A152,\x22def\x22\x3A23\x7D,\x22ppda_allowed\x22\x3A\x7B\x22att\x22\x3A292,\x22def\x22\x3A25\x7D,\x22deep\x22\x3A11,\x22deep_allowed\x22\x3A3,\x22scored\x22\x3A0,\x22missed\x22\x3A2,\x22xpts\x22\x3A1.15669999999999983941734171821735799312591552734375,\x22result\x22\x3A\x22l\x22,\x22date\x22\x3A\x222022\x2D08\x2D06\x2014\x3A00\x3A00\x22,\x22wins\x22\x3A0,\x22draws\x22\x3A0,\x22loses\x22\x3A1,\x22pts\x22\x3A0,\x22npxGD\x22\x3A\x2D0.099445999999999978857800897458218969404697418212890625\x7D,\x7B\x22h_a

In [33]:
# Everyone loves Arsenal so lets start by getting their info: 83
# convert JSON data into Python dictionary
data = json.loads(json_data)
print(data.keys()) # This prints all the keys for the teams in the league
print('='*50)
print(data['83'].keys()) # 83 is the key for arsenal
print('='*50)
print(data['83']['id'])
print('='*50)
print(data['83']['title'])
print('='*50)
print(data['83']['history'][0])

# Print pretty JSON data to check out what we have there
# s = json.dumps(data, indent=4, sort_keys=True)
# print(s)

dict_keys(['71', '72', '73', '74', '75', '78', '80', '81', '82', '83', '86', '87', '88', '89', '220', '228', '229', '244', '245', '249'])
dict_keys(['id', 'title', 'history'])
83
Arsenal
{'h_a': 'a', 'xG': 1.43601, 'xGA': 1.20637, 'npxG': 1.43601, 'npxGA': 1.20637, 'ppda': {'att': 355, 'def': 28}, 'ppda_allowed': {'att': 246, 'def': 24}, 'deep': 13, 'deep_allowed': 8, 'scored': 2, 'missed': 0, 'xpts': 1.5583999999999998, 'result': 'w', 'date': '2022-08-05 19:00:00', 'wins': 1, 'draws': 0, 'loses': 0, 'pts': 3, 'npxGD': 0.22964000000000007}


In [34]:
# Get teams and their relevant ids and put them into separate dictionary
teams = {}
for id in data.keys():
  teams[id] = data[id]['title']

print(teams)

{'71': 'Aston Villa', '72': 'Everton', '73': 'Bournemouth', '74': 'Southampton', '75': 'Leicester', '78': 'Crystal Palace', '80': 'Chelsea', '81': 'West Ham', '82': 'Tottenham', '83': 'Arsenal', '86': 'Newcastle United', '87': 'Liverpool', '88': 'Manchester City', '89': 'Manchester United', '220': 'Brighton', '228': 'Fulham', '229': 'Wolverhampton Wanderers', '244': 'Brentford', '245': 'Leeds', '249': 'Nottingham Forest'}


In [39]:
# EDA to get a feeling of how the JSON is structured
# Column names are all the same, so we just use first element
columns = []
# Check the sample of values per each column
values = []
for id in data.keys():
  columns = list(data[id]['history'][0].keys())
  values = list(data[id]['history'][0].values())
  break

print(columns)
print(values)

['h_a', 'xG', 'xGA', 'npxG', 'npxGA', 'ppda', 'ppda_allowed', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'result', 'date', 'wins', 'draws', 'loses', 'pts', 'npxGD']
['a', 0.488895, 0.588341, 0.488895, 0.588341, {'att': 152, 'def': 23}, {'att': 292, 'def': 25}, 11, 3, 0, 2, 1.1566999999999998, 'l', '2022-08-06 14:00:00', 0, 0, 1, 0, -0.09944599999999998]


In [42]:
# Setting variables and df for arsenal

arsenal_data = []
for row in data['83']['history']:
  arsenal_data.append(list(row.values()))
  
df = pd.DataFrame(arsenal_data, columns=columns)
df.head(2)


,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD
0,a,1.43601,1.206370,1.43601,1.206370,"{'att': 355, 'def': 28}","{'att': 246, 'def': 24}",13,8,2,0,1.5584,w,2022-08-05 19:00:00,1,0,0,3,0.229640
1,h,3.01070,0.690679,3.01070,0.690679,"{'att': 279, 'def': 27}","{'att': 196, 'def': 14}",10,5,4,2,2.7487,w,2022-08-13 14:00:00,1,0,0,3,2.320021


In [43]:
# Getting data for all teams
dataframes = {}
for id, team in teams.items():
  teams_data = []
  for row in data[id]['history']:
    teams_data.append(list(row.values()))
    
  df = pd.DataFrame(teams_data, columns=columns)
  dataframes[team] = df
  print('Added data for {}.'.format(team))

Added data for Aston Villa.
Added data for Everton.
Added data for Bournemouth.
Added data for Southampton.
Added data for Leicester.
Added data for Crystal Palace.
Added data for Chelsea.
Added data for West Ham.
Added data for Tottenham.
Added data for Arsenal.
Added data for Newcastle United.
Added data for Liverpool.
Added data for Manchester City.
Added data for Manchester United.
Added data for Brighton.
Added data for Fulham.
Added data for Wolverhampton Wanderers.
Added data for Brentford.
Added data for Leeds.
Added data for Nottingham Forest.


In [44]:
# Sample check of our newly created DataFrame
dataframes['Arsenal'].head(5)

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD
0,a,1.43601,1.206370,1.43601,1.206370,"{'att': 355, 'def': 28}","{'att': 246, 'def': 24}",13,8,2,0,1.5584,w,2022-08-05 19:00:00,1,0,0,3,0.229640
1,h,3.01070,0.690679,3.01070,0.690679,"{'att': 279, 'def': 27}","{'att': 196, 'def': 14}",10,5,4,2,2.7487,w,2022-08-13 14:00:00,1,0,0,3,2.320021
2,a,1.71081,0.211335,1.71081,0.211335,"{'att': 218, 'def': 25}","{'att': 360, 'def': 23}",8,1,3,0,2.6009,w,2022-08-20 16:30:00,1,0,0,3,1.499475
3,h,2.47427,0.926007,2.47427,0.926007,"{'att': 103, 'def': 19}","{'att': 327, 'def': 22}",17,1,2,1,2.4378,w,2022-08-27 16:30:00,1,0,0,3,1.548263
4,h,2.79971,0.297168,2.79971,0.297168,"{'att': 168, 'def': 17}","{'att': 278, 'def': 19}",18,3,2,1,2.8527,w,2022-08-31 18:30:00,1,0,0,3,2.502542


## Manipulating the data

In [46]:
# I do not think we need this? it creates some new columns 'ppda_coef' etc. 
for team, df in dataframes.items():
  dataframes[team]['ppda_coef'] = dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
  dataframes[team]['oppda_coef'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
  
# And check how our new dataframes look based on Sevilla dataframe
dataframes['Arsenal'].head(2)

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,...,xpts,result,date,wins,draws,loses,pts,npxGD,ppda_coef,oppda_coef
0,a,1.43601,1.206370,1.43601,1.206370,"{'att': 355, 'def': 28}","{'att': 246, 'def': 24}",13,8,2,...,1.5584,w,2022-08-05 19:00:00,1,0,0,3,0.229640,12.678571,10.25
1,h,3.01070,0.690679,3.01070,0.690679,"{'att': 279, 'def': 27}","{'att': 196, 'def': 14}",10,5,4,...,2.7487,w,2022-08-13 14:00:00,1,0,0,3,2.320021,10.333333,14.00


In [47]:
cols_to_sum = ['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'wins', 'draws', 'loses', 'pts', 'npxGD']
cols_to_mean = ['ppda_coef', 'oppda_coef']

In [48]:
frames = []
for team, df in dataframes.items():
  sum_data = pd.DataFrame(df[cols_to_sum].sum()).transpose()
  mean_data = pd.DataFrame(df[cols_to_mean].mean()).transpose()
  final_df = sum_data.join(mean_data)
  final_df['team'] = team
  final_df['matches'] = len(df)
  frames.append(final_df)
  
full_stat = pd.concat(frames)

In [49]:
# Next we reorder columns for better readability, sort rows based on points, reset index and add column 'position'.
full_stat = full_stat[['team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'npxG', 'xGA', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts']]
full_stat.sort_values('pts', ascending=False, inplace=True)
full_stat.reset_index(inplace=True, drop=True)
full_stat['position'] = range(1,len(full_stat)+1)

In [50]:
# Also in the original table we have values of differences between expected metrics and real. Let's add those too.
full_stat['xG_diff'] = full_stat['xG'] - full_stat['scored']
full_stat['xGA_diff'] = full_stat['xGA'] - full_stat['missed']
full_stat['xpts_diff'] = full_stat['xpts'] - full_stat['pts']

In [51]:
# Converting floats to integers where apropriate
cols_to_int = ['wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'deep', 'deep_allowed']
full_stat[cols_to_int] = full_stat[cols_to_int].astype(int)

In [52]:
# Prettifying output and final view of a DataFrame
col_order = ['position','team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'xG_diff', 'npxG', 'xGA', 'xGA_diff', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts', 'xpts_diff']
full_stat = full_stat[col_order]
pd.options.display.float_format = '{:,.2f}'.format
full_stat.head(10)

,position,team,matches,wins,draws,loses,scored,missed,pts,xG,...,xGA,xGA_diff,npxGA,npxGD,ppda_coef,oppda_coef,deep,deep_allowed,xpts,xpts_diff
0,1,Arsenal,13,11,1,1,31,11,34,27.26,...,11.30,0.30,9.78,16.73,9.73,14.95,154,54,28.43,-5.57
1,2,Manchester City,14,10,2,2,40,14,32,30.74,...,13.16,-0.84,11.64,16.82,12.26,30.43,193,44,30.94,-1.06
2,3,Newcastle United,15,8,6,1,29,11,30,28.43,...,17.45,6.45,16.69,10.21,10.40,12.96,158,90,26.32,-3.68
3,4,Tottenham,15,9,2,4,31,21,29,26.09,...,18.07,-2.93,17.30,6.50,14.63,17.10,107,114,25.89,-3.11
4,5,Manchester United,13,7,2,4,18,19,23,18.47,...,15.64,-3.36,14.88,2.83,13.72,14.39,108,106,19.96,-3.04
5,6,Liverpool,14,6,4,4,28,17,22,29.58,...,23.10,6.10,20.81,8.77,11.41,22.58,165,97,22.44,0.44
6,7,Brighton,13,6,3,4,22,17,21,24.07,...,14.91,-2.09,12.63,9.15,11.88,16.66,121,83,24.56,3.56
7,8,Chelsea,14,6,3,5,17,17,21,16.81,...,21.87,4.87,21.87,-6.58,9.47,16.44,100,62,15.72,-5.28
8,9,Crystal Palace,14,5,4,5,15,18,19,15.30,...,20.26,2.26,20.26,-6.63,18.73,10.15,63,133,15.87,-3.13
9,10,Brentford,15,4,7,4,23,25,19,23.45,...,22.63,-2.37,21.87,-1.47,12.50,10.63,88,139,22.09,3.09


## Scrapping the data for all the teams

In [22]:
season_data = dict()
season_data[seasons[4]] = full_stat
print(season_data)
full_data = dict()
full_data[leagues[0]] = season_data
print(full_data)

{'2022':     position             team  matches  wins  draws  loses  scored  missed  \
0          1      Real Madrid       12    10      2      0      29      10   
1          2        Barcelona       12    10      1      1      29       4   
2          3  Atletico Madrid       12     7      2      3      20      12   
3          4       Real Betis       12     7      2      3      16       8   
4          5    Real Sociedad       12     7      1      4      16      15   
5          6    Athletic Club       12     6      3      3      20      12   
6          7          Osasuna       12     6      2      4      13      11   
7          8   Rayo Vallecano       12     5      3      4      17      14   
8          9       Villarreal       12     5      3      4      14       8   
9         10         Valencia       12     4      3      5      18      14   
10        11  Real Valladolid       12     4      2      6      11      17   
11        12           Getafe       12     3      4    

## Loops to get all the data

In [23]:
full_data = dict()
for league in leagues:
  
  season_data = dict()
  for season in seasons:    
    url = base_url+'/'+league+'/'+season
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "lxml")

    # Based on the structure of the webpage, I found that data is in the JSON variable, under <script> tags
    scripts = soup.find_all('script')
    
    string_with_json_obj = ''

    # Find data for teams
    for el in scripts:
        if 'teamsData' in str(el):
          string_with_json_obj = str(el).strip()

    # print(string_with_json_obj)

    # strip unnecessary symbols and get only JSON data
    ind_start = string_with_json_obj.index("('")+2
    ind_end = string_with_json_obj.index("')")
    json_data = string_with_json_obj[ind_start:ind_end]
    json_data = json_data.encode('utf8').decode('unicode_escape')
    
    
    # convert JSON data into Python dictionary
    data = json.loads(json_data)
    
    # Get teams and their relevant ids and put them into separate dictionary
    teams = {}
    for id in data.keys():
      teams[id] = data[id]['title']
      
    # EDA to get a feeling of how the JSON is structured
    # Column names are all the same, so we just use first element
    columns = []
    # Check the sample of values per each column
    values = []
    for id in data.keys():
      columns = list(data[id]['history'][0].keys())
      values = list(data[id]['history'][0].values())
      break
      
    # Getting data for all teams
    dataframes = {}
    for id, team in teams.items():
      teams_data = []
      for row in data[id]['history']:
        teams_data.append(list(row.values()))

      df = pd.DataFrame(teams_data, columns=columns)
      dataframes[team] = df
      # print('Added data for {}.'.format(team))
      
    
    for team, df in dataframes.items():
      dataframes[team]['ppda_coef'] = dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
      dataframes[team]['oppda_coef'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
      
    cols_to_sum = ['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'wins', 'draws', 'loses', 'pts', 'npxGD']
    cols_to_mean = ['ppda_coef', 'oppda_coef']
    
    frames = []
    for team, df in dataframes.items():
      sum_data = pd.DataFrame(df[cols_to_sum].sum()).transpose()
      mean_data = pd.DataFrame(df[cols_to_mean].mean()).transpose()
      final_df = sum_data.join(mean_data)
      final_df['team'] = team
      final_df['matches'] = len(df)
      frames.append(final_df)

    full_stat = pd.concat(frames)
    
    full_stat = full_stat[['team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'npxG', 'xGA', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts']]
    full_stat.sort_values('pts', ascending=False, inplace=True)
    full_stat.reset_index(inplace=True, drop=True)
    full_stat['position'] = range(1,len(full_stat)+1)
    
    full_stat['xG_diff'] = full_stat['xG'] - full_stat['scored']
    full_stat['xGA_diff'] = full_stat['xGA'] - full_stat['missed']
    full_stat['xpts_diff'] = full_stat['xpts'] - full_stat['pts']
    
    cols_to_int = ['wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'deep', 'deep_allowed']
    full_stat[cols_to_int] = full_stat[cols_to_int].astype(int)
    
    col_order = ['position', 'team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'xG_diff', 'npxG', 'xGA', 'xGA_diff', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts', 'xpts_diff']
    full_stat = full_stat[col_order]
    full_stat = full_stat.set_index('position')
    # print(full_stat.head(20))
    
    season_data[season] = full_stat
  
  df_season = pd.concat(season_data)
  full_data[league] = df_season
  
data = pd.concat(full_data)
data.head()

team  matches  wins  draws  loses  scored  \
             position                                                         
La_liga 2018 1               Barcelona       38    26      9      3      90   
             2         Atletico Madrid       38    22     10      6      55   
             3             Real Madrid       38    21      5     12      63   
             4                Valencia       38    15     16      7      51   
             5                 Sevilla       38    17      8     13      62   

                       missed  pts    xG  xG_diff  ...   xGA  xGA_diff  npxGA  \
             position                              ...                          
La_liga 2018 1             36   87 83.28    -6.72  ... 44.93      8.93  43.44   
             2             29   76 51.87    -3.13  ... 41.43     12.43  37.72   
             3             46   68 68.65     5.65  ... 48.68      2.68  42.73   
             4             35   61 61.88    10.88  ... 42.85      7.85  36.91   
             5             47   59 69.16     7.16  ... 46.71     -0.29  41.51   

                       npxGD  ppda_coef  oppda_coef  deep  deep_allowed  xpts  \
             position                                                           
La_liga 2018 1         33.14       9.02       16.40   417           171 73.96   
             2         11.01      11.07       11.10   252           190 59.43   
             3         19.24       8.90       14.78   341           168 64.77   
             4         19.66      12.96        9.47   278           215 65.16   
             5         23.03      10.65       10.02   321           211 65.08   

                       xpts_diff  
             position             
La_liga 2018 1            -13.04  
             2            -16.57  
             3             -3.23  
             4              4.16  
             5              6.08  

[5 rows x 21 columns]

## Cleaning up the table
We only want to keep the following columns:
* Position
* Team
* Matches
* Wins
* Draws
* Loses
* Points

In [24]:
data1 = data.drop(['scored', 'missed', 'xG', 'xG_diff','npxG','npxGA','xGA','xGA_diff', 'npxGD', 'ppda_coef','oppda_coef','deep', 'deep_allowed','xpts', 'xpts_diff'], axis=1)
data1

team  matches  wins  draws  loses  pts
             position                                                    
La_liga 2018 1                Barcelona       38    26      9      3   87
             2          Atletico Madrid       38    22     10      6   76
             3              Real Madrid       38    21      5     12   68
             4                 Valencia       38    15     16      7   61
             5                  Sevilla       38    17      8     13   59
...                                 ...      ...   ...    ...    ...  ...
RFPL    2022 12         Nizhny Novgorod       15     3      4      8   13
             13        Lokomotiv Moscow       15     3      3      9   12
             14                   Fakel       15     1      9      5   12
             15                  Khimki       15     2      3     10    9
             16          Torpedo Moscow       15     1      3     11    6

[570 rows x 6 columns]

In [25]:
# We only want EPL and 2022
data2 = data1.filter(like='EPL', axis=0)
data2

team  matches  wins  draws  loses  pts
         position                                                           
EPL 2018 1                 Manchester City       38    32      2      4   98
         2                       Liverpool       38    30      7      1   97
         3                         Chelsea       38    21      9      8   72
         4                       Tottenham       38    23      2     13   71
         5                         Arsenal       38    21      7     10   70
...                                    ...      ...   ...    ...    ...  ...
    2022 16                    Aston Villa       13     3      3      7   12
         17                    Southampton       13     3      3      7   12
         18                      Leicester       13     3      2      8   11
         19        Wolverhampton Wanderers       13     2      4      7   10
         20              Nottingham Forest       13     2      3      8    9

[100 rows x 6 columns]

In [26]:
data3=data2.filter(like='2022', axis=0)
data3

team  matches  wins  draws  loses  pts
         position                                                           
EPL 2022 1                         Arsenal       12    10      1      1   31
         2                 Manchester City       12     9      2      1   29
         3                       Tottenham       13     8      2      3   26
         4                Newcastle United       13     6      6      1   24
         5               Manchester United       12     7      2      3   23
         6                         Chelsea       12     6      3      3   21
         7                          Fulham       13     5      4      4   19
         8                        Brighton       12     5      3      4   18
         9                  Crystal Palace       12     4      4      4   16
         10                      Liverpool       12     4      4      4   16
         11                      Brentford       13     3      6      4   15
         12                       West Ham       13     4      2      7   14
         13                        Everton       13     3      5      5   14
         14                    Bournemouth       13     3      4      6   13
         15                          Leeds       12     3      3      6   12
         16                    Aston Villa       13     3      3      7   12
         17                    Southampton       13     3      3      7   12
         18                      Leicester       13     3      2      8   11
         19        Wolverhampton Wanderers       13     2      4      7   10
         20              Nottingham Forest       13     2      3      8    9

In [27]:
# Converting the data frame to an HTML Table
html_table = data3.to_html()
print(html_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th></th>
      <th>team</th>
      <th>matches</th>
      <th>wins</th>
      <th>draws</th>
      <th>loses</th>
      <th>pts</th>
    </tr>
    <tr>
      <th></th>
      <th></th>
      <th>position</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="20" valign="top">EPL</th>
      <th rowspan="20" valign="top">2022</th>
      <th>1</th>
      <td>Arsenal</td>
      <td>12</td>
      <td>10</td>
      <td>1</td>
      <td>1</td>
      <td>31</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Manchester City</td>
      <td>12</td>
      <td>9</td>
      <td>2</td>
      <td>1</td>
      <td>29</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Tottenham</td>
      <td>13</td>
      <td>8</td>
      <td>2</td>
      <td>3</td>
      <td>26</td>
    </tr>
   

In [30]:

# write html to file
text_file = open("index.html", "w")
text_file.write(html_table)
text_file.close()

In [32]:
import webbrowser
url = 'index.html'
webbrowser.open(url, new=2)  # open in new tab

True